In [1]:
import matplotlib.pyplot as plt
import numpy as np

from lmfit import CompositeModel, Model
from lmfit.lineshapes import gaussian, step

# create data from broadened step
npts = 201
x = np.linspace(0, 10, npts)
y = step(x, amplitude=12.5, center=4.5, sigma=0.88, form='erf')
np.random.seed(0)
y = y + np.random.normal(size=npts, scale=0.35)


def jump(x, mid):
    """heaviside step function"""
    o = np.zeros(len(x))
    imid = max(np.where(x <= mid)[0])
    o[imid:] = 1.0
    return o


def convolve(arr, kernel):
    """simple convolution of two arrays"""
    npts = min(len(arr), len(kernel))
    pad = np.ones(npts)
    tmp = np.concatenate((pad*arr[0], arr, pad*arr[-1]))
    out = np.convolve(tmp, kernel, mode='valid')
    noff = int((len(out) - npts) / 2)
    return out[noff:noff+npts]


# create Composite Model using the custom convolution operator
mod = CompositeModel(Model(jump), Model(gaussian), convolve)

pars = mod.make_params(amplitude=1, center=3.5, sigma=1.5, mid=5.0)

# 'mid' and 'center' should be completely correlated, and 'mid' is
# used as an integer index, so a very poor fit variable:
pars['mid'].vary = False

# fit this model to data array y
result = mod.fit(y, params=pars, x=x)

print(result.fit_report())

plot_components = False

# plot results
plt.plot(x, y, 'bo')
if plot_components:
    # generate components
    comps = result.eval_components(x=x)
    plt.plot(x, 10*comps['jump'], 'k--')
    plt.plot(x, 10*comps['gaussian'], 'r-')
else:
    plt.plot(x, result.init_fit, 'k--')
    plt.plot(x, result.best_fit, 'r-')
plt.show()
# <end examples/doc_model_composite.py>


[[Model]]
    (Model(jump) <function convolve at 0x113631268> Model(gaussian))
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 25
    # data points      = 201
    # variables        = 3
    chi-square         = 24.7562335
    reduced chi-square = 0.12503148
    Akaike info crit   = -414.939746
    Bayesian info crit = -405.029832
[[Variables]]
    mid:        5 (fixed)
    sigma:      0.59576118 +/- 0.01348582 (2.26%) (init = 1.5)
    center:     4.50853671 +/- 0.00973231 (0.22%) (init = 3.5)
    amplitude:  0.62508459 +/- 0.00189732 (0.30%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(center, amplitude) =  0.329
    C(sigma, amplitude)  =  0.268



<Figure size 640x480 with 1 Axes>

In [2]:
result.params

AttributeError: 'Parameter' object has no attribute 'user_value'

Parameters([('mid', <Parameter 'mid', value=5.0 (fixed), bounds=[-inf:inf]>),
            ('sigma',
             <Parameter 'sigma', value=0.595761182715068 +/- 0.0135, bounds=[-inf:inf]>),
            ('center',
             <Parameter 'center', value=4.508536711163319 +/- 0.00973, bounds=[-inf:inf]>),
            ('amplitude',
             <Parameter 'amplitude', value=0.6250845856983748 +/- 0.0019, bounds=[-inf:inf]>)])

In [3]:
result